In [5]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import pandas as pd
df= pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/updated_cleaned_data.csv")
df.head()

,document,summary,id,rg_labels,ext_labels
0,hi i I am getting a new laptop i do nt know wh...,of my post is asking what can i do compensate ...,TLDR_RS_2021-04-cm-17760.json,[1.0],[1]
1,it 's possible that many of you already do thi...,if you play hearthstone on mac and it runs poo...,TLDR_RS_2021-02-cm-3714.json,"[0.16403576510432702, 0.35967370810124305, 0.2...","[0, 1, 0, 0]"
2,"i will be the first to admit , i 've got a wei...",i 'm tired of my boyfriend bringing up my weig...,TLDR_RS_2021-04-cm-1732.json,"[0.1865050509103, 0.389592934910708, 0.2526842...","[0, 1, 0, 0, 0]"
3,"hello , my friend 22m broke up a few months ag...",: my friend has suddenly stopped talking to me...,TLDR_RS_2021-03-cm-34926.json,"[0.13981801700034802, 0.054837487497308005, 0....","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]"
4,background : my 18f ex boyfriend 18m really wa...,my ex is very erratic and depressed and never ...,TLDR_RS_2021-02-cm-18586.json,"[0.18652821177119003, 0.149288690725626, 0.038...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]"


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   document    50000 non-null  object
 1   summary     49993 non-null  object
 2   id          50000 non-null  object
 3   rg_labels   50000 non-null  object
 4   ext_labels  50000 non-null  object
dtypes: object(5)
memory usage: 1.9+ MB


In [9]:
model_df = df.copy()

# specifying the relevant columns in the new dataset
relevant_columns = ['document', 'summary']

# filtering the DataFrame to keep only the relevant columns
model_df = model_df[relevant_columns]

# displaying the first few rows of the new DataFrame
model_df.head()

,document,summary
0,hi i I am getting a new laptop i do nt know wh...,of my post is asking what can i do compensate ...
1,it 's possible that many of you already do thi...,if you play hearthstone on mac and it runs poo...
2,"i will be the first to admit , i 've got a wei...",i 'm tired of my boyfriend bringing up my weig...
3,"hello , my friend 22m broke up a few months ag...",: my friend has suddenly stopped talking to me...
4,background : my 18f ex boyfriend 18m really wa...,my ex is very erratic and depressed and never ...


In [10]:
# importing all the necessary libraries
import torch
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import train_test_split
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments, EarlyStoppingCallback
from torch.utils.data import DataLoader

In [11]:
# extracting LDA topics using TF-IDF
# cleaning and preparing the documents for LDA
model_df['document'] = model_df['document'].fillna('').astype(str)
tfidf_vectorizer = TfidfVectorizer(max_features=7000, stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(model_df['document'])

# running LDA on documents
lda_model = LatentDirichletAllocation(n_components=10, random_state=42, max_iter=10)
lda_topics = lda_model.fit_transform(tfidf_matrix)

# mapping topics back to words
topic_keywords = []
for topic_idx, topic in enumerate(lda_model.components_):
    top_keywords = [tfidf_vectorizer.get_feature_names_out()[i] for i in topic.argsort()[-10:]]
    topic_keywords.append(f"Topic {topic_idx}: " + " ".join(top_keywords))

print("\n".join(topic_keywords))

Topic 0: friends said really know feel did relationship like just nt
Topic 1: hair little ve did cat got water dog just like
Topic 2: really work job ve know people feel school just like
Topic 3: screen issue laptop like new ve just using use pc
Topic 4: know game world think story characters people just character like
Topic 5: time family nt car home did just house dad mom
Topic 6: team player damage playing like just players games play game
Topic 7: buy short x200b company money stock gme shares market price
Topic 8: eu si para di sa en se da la que
Topic 9: did weight ive sleep mg doctor day like just pain


In [12]:
# Function to extract top keywords for a given topic
def get_topic_keywords(lda_model, tfidf_vectorizer, n_top_words=10):
    keywords = []
    for topic_idx, topic in enumerate(lda_model.components_):
        top_keywords = [tfidf_vectorizer.get_feature_names_out()[i] for i in topic.argsort()[-n_top_words:]]
        keywords.append(top_keywords)
    return keywords

# extracting topic keywords
topic_keywords_list = get_topic_keywords(lda_model, tfidf_vectorizer)

# adding the top keywords from the most probable topic to the document
def add_keywords_to_document(row, topic_keywords_list):
    most_probable_topic = row['topics']
    keywords = " ".join(topic_keywords_list[int(most_probable_topic)])                                                      # joining top keywords of the topic
    return row['document'] + " " + keywords

# mapping the most probable topic keywords to each document
model_df['topics'] = lda_topics.argmax(axis=1)
model_df['new_document'] = model_df.apply(add_keywords_to_document, axis=1, args=(topic_keywords_list,))

In [13]:
model_df.head()

,document,summary,topics,new_document
0,hi i I am getting a new laptop i do nt know wh...,of my post is asking what can i do compensate ...,3,hi i I am getting a new laptop i do nt know wh...
1,it 's possible that many of you already do thi...,if you play hearthstone on mac and it runs poo...,3,it 's possible that many of you already do thi...
2,"i will be the first to admit , i 've got a wei...",i 'm tired of my boyfriend bringing up my weig...,0,"i will be the first to admit , i 've got a wei..."
3,"hello , my friend 22m broke up a few months ag...",: my friend has suddenly stopped talking to me...,0,"hello , my friend 22m broke up a few months ag..."
4,background : my 18f ex boyfriend 18m really wa...,my ex is very erratic and depressed and never ...,0,background : my 18f ex boyfriend 18m really wa...


In [14]:
import torch

# checking if GPU is available and set the device
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Using device: {device}")

Using device: cuda


In [22]:
from sklearn.model_selection import train_test_split
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments
import torch
import pandas as pd

# selecting a smaller subset of the dataset
subset_size = 10000                                                                   # choosing the size of your subset
subset_df = model_df.sample(n=subset_size, random_state=42)   # randomly selecting records

# splitting the dataset into training and testing sets
train_df, test_df = train_test_split(subset_df, test_size=0.2, random_state=42)

# loading the BART tokenizer
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')

# Function to tokenize the text data
def tokenize_function(examples, summary_max_length=150):
    # converting pandas series to list of strings
    documents = examples['new_document'].tolist()  # ensuring documents are a list of strings
    summaries = examples['summary'].tolist()    # ensuring summaries are a list of strings

    # tokenizing the documents and summaries
    model_inputs = tokenizer(documents, max_length=1024, truncation=True, padding=True, return_tensors="pt")
    labels = tokenizer(summaries, max_length=summary_max_length, truncation=True, padding=True, return_tensors="pt")

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# cleaning the data by ensuring there are no NaN values and the data is in string format
train_df['new_document'] = train_df['new_document'].fillna('').astype(str)
train_df['summary'] = train_df['summary'].fillna('').astype(str)

# applying the tokenization function to the DataFrame
train_encodings = tokenize_function(train_df, summary_max_length=150)
test_encodings = tokenize_function(test_df, summary_max_length=150)

In [23]:
from transformers import BartForConditionalGeneration

# loading pre-trained BART model
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')

# freezing the first 10 encoder layers
for layer in model.model.encoder.layers[:10]:
    for param in layer.parameters():
        param.requires_grad = False

# freezing the first 10 decoder layers
for layer in model.model.decoder.layers[:10]:
    for param in layer.parameters():
        param.requires_grad = False

In [24]:
from transformers import AdamW

# moving the model to the selected device
model.to(device)

# updating the SummaryDataset to move tensors to the correct device
class SummaryDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, device):
        self.encodings = encodings
        self.device = device

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]).to(self.device) for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

train_dataset = SummaryDataset(train_encodings, device)
test_dataset = SummaryDataset(test_encodings, device)

model.safetensors:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

In [25]:
from torch.utils.data import DataLoader

# When creating the DataLoader, set pin_memory=False if you're using GPU
train_loader = DataLoader(train_dataset, batch_size=8, pin_memory=False)
test_loader = DataLoader(test_dataset, batch_size=8, pin_memory=False)

In [26]:
from transformers import EarlyStoppingCallback

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='steps',
    eval_steps=500,
    learning_rate=5e-7,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True,
    logging_dir='./logs',
    save_steps=500,
    logging_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',
    dataloader_pin_memory=False  # Ensure this is set to False
)

# initializing trainer with EarlyStoppingCallback
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

In [27]:
# training the model
trainer.train()

Step,Training Loss,Validation Loss
500,1.008700,1.021202
1000,1.047300,0.997157
1500,0.936800,0.992309
2000,0.962800,0.988123
2500,0.939700,0.987246
3000,0.938400,0.985304


There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=3000, training_loss=1.1408707707722983, metrics={'train_runtime': 2915.7062, 'train_samples_per_second': 8.231, 'train_steps_per_second': 1.029, 'total_flos': 5.2010510450688e+16, 'train_loss': 1.1408707707722983, 'epoch': 3.0})

In [28]:
# Save the model
model.save_pretrained('/content/drive/MyDrive/bart_model')

# Save the tokenizer
tokenizer.save_pretrained('/content/drive/MyDrive/bart_tokenizer')

('/content/drive/MyDrive/bart_tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/bart_tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/bart_tokenizer/vocab.json',
 '/content/drive/MyDrive/bart_tokenizer/merges.txt',
 '/content/drive/MyDrive/bart_tokenizer/added_tokens.json')

In [29]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [30]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=74800150e259dad8fad4482e8391be9a5a0fe5e6284ecfb873228466ef62329e
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [33]:
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu
import nltk
from sklearn.metrics import f1_score
from transformers import BartTokenizer, BertTokenizer

# Download necessary NLTK data files
nltk.download('punkt')

# Initialize ROUGE scorer
rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Define device and move model to that device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Initialize BART tokenizer (consistent with the one used for model)
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')

# Initialize BERT tokenizer for F1 calculation
bert_tokenizer = BertTokenizer.from_pretrained("bert-large-cased")

# Function to generate summaries
# Function to generate summaries
def generate_summary(text, max_length=150):
    model.eval()  # Set model to evaluation mode

    # Tokenize the input and move to the correct device
    inputs = tokenizer(text, max_length=512, truncation=True, return_tensors="pt").to(device)

    # Generate summary
    summary_ids = model.generate(inputs['input_ids'], max_length=max_length, num_beams=4, length_penalty=2.0)

    # Decode the generated summary and return
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Calculate ROUGE, BLEU, and BERT F1 scores
rouge_scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}
bleu_scores = []
bert_f1_scores = []

for idx, row in test_df.iterrows():
    # Generate model summary
    generated_summary = generate_summary(row['document'])

    # Reference summary
    reference_summary = row['summary']

    # Calculate ROUGE scores
    scores = rouge_scorer.score(reference_summary, generated_summary)
    rouge_scores['rouge1'].append(scores['rouge1'].fmeasure)
    rouge_scores['rouge2'].append(scores['rouge2'].fmeasure)
    rouge_scores['rougeL'].append(scores['rougeL'].fmeasure)

    # Calculate BLEU score
    reference_tokens = nltk.word_tokenize(reference_summary)
    generated_tokens = nltk.word_tokenize(generated_summary)
    bleu_score = sentence_bleu([reference_tokens], generated_tokens)
    bleu_scores.append(bleu_score)

    # Tokenize using BERT tokenizer (to compare token IDs, not raw words)
    reference_tokens_bert = bert_tokenizer.encode(reference_summary, add_special_tokens=False)
    generated_tokens_bert = bert_tokenizer.encode(generated_summary, add_special_tokens=False)

    # Calculate BERT F1 score
    common_tokens = set(reference_tokens_bert) & set(generated_tokens_bert)
    precision = len(common_tokens) / len(generated_tokens_bert) if len(generated_tokens_bert) > 0 else 0
    recall = len(common_tokens) / len(reference_tokens_bert) if len(reference_tokens_bert) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    bert_f1_scores.append(f1)

# Calculate average scores
avg_rouge1 = sum(rouge_scores['rouge1']) / len(rouge_scores['rouge1'])
avg_rouge2 = sum(rouge_scores['rouge2']) / len(rouge_scores['rouge2'])
avg_rougeL = sum(rouge_scores['rougeL']) / len(rouge_scores['rougeL'])
avg_bleu = sum(bleu_scores) / len(bleu_scores)
avg_bert_f1 = sum(bert_f1_scores) / len(bert_f1_scores)

# Print evaluation results
print(f"Average ROUGE-1 Score: {avg_rouge1:.4f}")
print(f"Average ROUGE-2 Score: {avg_rouge2:.4f}")
print(f"Average ROUGE-L Score: {avg_rougeL:.4f}")
print(f"Average BLEU Score: {avg_bleu:.4f}")
print(f"Average BERT F1 Score: {avg_bert_f1:.4f}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Token indices sequence length is longer than the specified maximum sequence length for this model (561 > 512). Running this sequence through the model will result in indexing errors


Average ROUGE-1 Score: 0.2648
Average ROUGE-2 Score: 0.0782
Average ROUGE-L Score: 0.1917
Average BLEU Score: 0.0217
Average BERT F1 Score: 0.2416
